In [1]:
# ================================
# Etapa 0: Configurações de execução
# ================================
from pyspark.sql.functions import col, when, to_date, lit

# Rebase para datas antigas
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInWrite", "CORRECTED")

# Adaptive Query Execution (melhora planejamento de joins e skew)
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", "true")
spark.conf.set("spark.sql.adaptive.skewJoin.enabled", "true")

# Partições (ajuste conforme seu cluster/dados)
spark.conf.set("spark.sql.shuffle.partitions", "400")

# Caminho físico do lakehouse de destino
path_destino = "abfss://ws_departamento_pessoal@onelake.dfs.fabric.microsoft.com/lk_departamento_pessoal.Lakehouse/Tables/tab_gold_dim_funcionario_historico"

# ================================
# Etapa 1: Leitura(s) necessárias
# ================================
ahstutilizaponto = spark.read.format("delta").load(
    "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_AutomacaoPonto.Lakehouse/Tables/ahstutilizaponto"
)
ahstutilizaponto.createOrReplaceTempView("ahstutilizaponto")

# ================================
# Etapa 2: SQL otimizado (mesma lógica/campos)
# ================================
df_resultado = spark.sql("""
WITH 
-- Normalizamos datas no início para evitar CAST repetido nos JOINs
CTE_Situacao_Periodos AS (
    SELECT
        PF.CODCOLIGADA, 
        PF.CHAPA,
        PF.CODPESSOA,
        CAST(PF.DATAADMISSAO AS DATE) AS DATAADMISSAO,
        PF.TIPOADMISSAO,
        AD.DESCRICAO AS DESCRICAO_TIPO_ADMISSAO,
        CAST(PF.DATADEMISSAO AS DATE) AS DATADEMISSAO,
        PF.TIPODEMISSAO,
        DE.DESCRICAO AS DESCRICAO_TIPO_DEMISSAO,
        CAST(HS.DATAMUDANCA AS DATE) AS DATAMUDANCA, 
        HS.NOVASITUACAO,
        SIT.DESCRICAO,
        
        CASE
            WHEN HS.NOVASITUACAO = 'D' 
                THEN DATE_ADD(CAST(HS.DATAMUDANCA AS DATE), 1)
            ELSE COALESCE(
                CAST(LEAD(HS.DATAMUDANCA, 1) OVER (
                    PARTITION BY PF.CODCOLIGADA, PF.CHAPA ORDER BY HS.DATAMUDANCA
                ) AS DATE),
                current_date()
            )
        END AS PROXIMA_DATAMUDANCA
    FROM      pfunc AS PF
    INNER JOIN pfhstsit AS HS
           ON PF.CODCOLIGADA = HS.CODCOLIGADA AND PF.CHAPA = HS.CHAPA
    INNER JOIN pcodsituacao AS SIT 
           ON HS.NOVASITUACAO = SIT.CODCLIENTE     
    LEFT  JOIN ptpadmissao AS AD 
           ON PF.TIPOADMISSAO = AD.CODCLIENTE
    LEFT  JOIN ptpdemissao AS DE 
           ON PF.TIPODEMISSAO = DE.CODCLIENTE
),

CTE_Situacao_Diaria AS (
    
    SELECT
        CAST(DC.DATA AS DATE) AS DATA,
        P.CODCOLIGADA,
        P.CHAPA,
        P.CODPESSOA,
        P.DATAADMISSAO,
        P.TIPOADMISSAO,
        P.DESCRICAO_TIPO_ADMISSAO,
        P.DATADEMISSAO,
        P.TIPODEMISSAO,
        P.DESCRICAO_TIPO_DEMISSAO,
        P.DATAMUDANCA,
        P.PROXIMA_DATAMUDANCA,
        P.NOVASITUACAO AS SITUACAO,
        P.DESCRICAO AS DESCRICAO_SITUACAO
    FROM dcalendario AS DC
    JOIN CTE_Situacao_Periodos AS P
      ON CAST(DC.DATA AS DATE) >= P.DATAMUDANCA 
     AND CAST(DC.DATA AS DATE)  < P.PROXIMA_DATAMUDANCA
),

CTE_Hist_Funcao AS (
    SELECT DISTINCT 
        HF.CODCOLIGADA, 
        HF.CHAPA, 
        HF.CODFUNCAO AS FUNCAO,
        PF.NOME AS DESCRICAO_FUNCAO,
        PF.CARGO, 
        CAST(HF.DTMUDANCA AS DATE) AS DTMUDANCA,
        COALESCE(
            CAST(LEAD(HF.DTMUDANCA, 1) OVER (
                PARTITION BY HF.CODCOLIGADA, HF.CHAPA ORDER BY HF.DTMUDANCA
            ) AS DATE),
            current_date()
        ) AS PROXIMA_DATAMUDANCA_FCO
    FROM pfhstfco AS HF
    INNER JOIN pfuncao AS PF 
            ON HF.CODCOLIGADA = PF.CODCOLIGADA AND HF.CODFUNCAO = PF.CODIGO 
),

CTE_Hist_Secao AS (
    SELECT DISTINCT 
        HS.CODCOLIGADA, 
        HS.CHAPA, 
        HS.CODSECAO AS SECAO, 
        CAST(HS.DTMUDANCA AS DATE) AS DTMUDANCA,
        COALESCE(
            CAST(LEAD(HS.DTMUDANCA, 1) OVER (
                PARTITION BY HS.CODCOLIGADA, HS.CHAPA ORDER BY HS.DTMUDANCA
            ) AS DATE),
            current_date()
        ) AS PROXIMA_DATAMUDANCA_SEC
    FROM pfhstsec AS HS
),

CTE_Hist_Salario AS (
    SELECT DISTINCT 
        HSA.CODCOLIGADA, 
        HSA.CHAPA, 
        HSA.SALARIO, 
        CAST(HSA.DTMUDANCA AS DATE) AS DTMUDANCA,
        COALESCE(
            CAST(LEAD(HSA.DTMUDANCA, 1) OVER (
                PARTITION BY HSA.CODCOLIGADA, HSA.CHAPA ORDER BY HSA.DTMUDANCA
            ) AS DATE),
            current_date()
        ) AS PROXIMA_DATAMUDANCA_SAL
    FROM pfhstsal AS HSA
),

CTE_Hist_Equipe AS (
    SELECT DISTINCT 
        HSE.CODCOLIGADA, 
        HSE.CHAPA, 
        HSE.CODEQUIPE AS EQUIPE,
        PE.DESCRICAO AS DESCRICAO_EQUIPE, 
        CAST(HSE.DTMUDANCA AS DATE) AS DTMUDANCA,
        COALESCE(
            CAST(LEAD(HSE.DTMUDANCA, 1) OVER (
                PARTITION BY HSE.CODCOLIGADA, HSE.CHAPA ORDER BY HSE.DTMUDANCA
            ) AS DATE),
            current_date()
        ) AS PROXIMA_DATAMUDANCA_EQP
    FROM zmd_hst_equipe AS HSE
    LEFT JOIN pequipe AS PE 
           ON HSE.CODEQUIPE = PE.CODCLIENTE
),

CTE_Hist_Ponto AS (
    SELECT DISTINCT 
        HPO.CODCOLIGADA, 
        HPO.CHAPA, 
        HPO.UTILIZA, 
        CAST(HPO.DATAINICIO AS DATE) AS DATAINICIO,
        COALESCE(
            CAST(LEAD(HPO.DATAINICIO, 1) OVER (
                PARTITION BY HPO.CODCOLIGADA, HPO.CHAPA ORDER BY HPO.DATAINICIO
            ) AS DATE),
            current_date()
        ) AS PROXIMA_DATAINICIO_PON
    FROM ahstutilizaponto AS HPO
)

SELECT 
    S.DATA,
    S.CODCOLIGADA,
    S.CHAPA,
    S.CODPESSOA,
    S.DATAADMISSAO,
    S.TIPOADMISSAO,
    S.DESCRICAO_TIPO_ADMISSAO,
    S.DATADEMISSAO,
    S.TIPODEMISSAO,
    S.DESCRICAO_TIPO_DEMISSAO,
    PP.NOME AS NOME_FUNCIONARIO,
    PP.CPF,
    CAST(PP.DTNASCIMENTO AS DATE) AS DTNASCIMENTO,
    PP.ESTADOCIVIL,
    PP.SEXO,
    CASE 
        WHEN (
            COALESCE(PP.DEFICIENTEAUDITIVO, 0) + 
            COALESCE(PP.DEFICIENTEFALA, 0) + 
            COALESCE(PP.DEFICIENTEFISICO, 0) + 
            COALESCE(PP.DEFICIENTEINTELECTUAL, 0) + 
            COALESCE(PP.DEFICIENTEMENTAL, 0) + 
            COALESCE(PP.DEFICIENTEMOBREDUZIDA, 0) + 
            COALESCE(PP.DEFICIENTEOBSERVACAO, 0) + 
            COALESCE(PP.DEFICIENTEVISUAL, 0)
        ) > 0 THEN 'Sim'
        ELSE 'Não'
    END AS PCD,
    PP.CIDADE,
    PP.ESTADO,
    PP.PAIS,
    PCI.DESCRICAO AS DESCRICAO_GRAU_INSTRUCAO,
    PN.DESCRICAO AS DESCRICAO_NACIONALIDADE,
    S.DATAMUDANCA,
    S.PROXIMA_DATAMUDANCA,
    S.SITUACAO,
    S.DESCRICAO_SITUACAO,
    FCO.FUNCAO,
    FCO.DESCRICAO_FUNCAO,
    FCO.CARGO,
    SEC.SECAO,
    EQP.EQUIPE,
    EQP.DESCRICAO_EQUIPE,
    CAST(
        CASE 
            WHEN EQP.EQUIPE IS NOT NULL THEN LEFT(EQP.EQUIPE, 4)
            ELSE SUBSTRING(SEC.SECAO, 11, 4)
        END AS STRING
    ) AS EQUIPE_CC,
    CASE 
        WHEN PON.UTILIZA = 0 THEN 'Não'
        WHEN PON.UTILIZA = 1 THEN 'Sim'
    END AS UTILIZA_PONTO, 
    SAL.SALARIO,
    ROW_NUMBER() OVER (
        PARTITION BY S.CODCOLIGADA, S.CODPESSOA, S.CHAPA 
        ORDER BY S.DATA DESC
    ) AS RK,
    ROW_NUMBER() OVER (
        PARTITION BY S.CODCOLIGADA, S.CODPESSOA, S.CHAPA, last_day(CAST(S.DATA AS DATE)) 
        ORDER BY S.DATA DESC
    ) AS RK_MES,
    MIN(S.DATA) OVER (
        PARTITION BY S.CODCOLIGADA, S.CHAPA, CONCAT(
            CASE WHEN EQP.EQUIPE IS NOT NULL THEN LEFT(EQP.EQUIPE, 4) ELSE SUBSTRING(SEC.SECAO, 11, 4) END,
            COALESCE(FCO.FUNCAO, '')
        )
    ) AS DATA_MINIMA_CC_FUNCAO,
    MAX(S.DATA) OVER (
        PARTITION BY S.CODCOLIGADA, S.CHAPA, CONCAT(
            CASE WHEN EQP.EQUIPE IS NOT NULL THEN LEFT(EQP.EQUIPE, 4) ELSE SUBSTRING(SEC.SECAO, 11, 4) END,
            COALESCE(FCO.FUNCAO, '')
        )
    ) AS DATA_MAXIMA_CC_FUNCAO
FROM      CTE_Situacao_Diaria AS S
LEFT JOIN CTE_Hist_Funcao AS FCO 
       ON S.CODCOLIGADA = FCO.CODCOLIGADA 
      AND S.CHAPA       = FCO.CHAPA 
      AND S.DATA       >= FCO.DTMUDANCA 
      AND S.DATA        < FCO.PROXIMA_DATAMUDANCA_FCO
LEFT JOIN CTE_Hist_Secao AS SEC
       ON S.CODCOLIGADA = SEC.CODCOLIGADA 
      AND S.CHAPA       = SEC.CHAPA 
      AND S.DATA       >= SEC.DTMUDANCA 
      AND S.DATA        < SEC.PROXIMA_DATAMUDANCA_SEC
LEFT JOIN CTE_Hist_Salario AS SAL 
       ON S.CODCOLIGADA = SAL.CODCOLIGADA 
      AND S.CHAPA       = SAL.CHAPA 
      AND S.DATA       >= SAL.DTMUDANCA 
      AND S.DATA        < SAL.PROXIMA_DATAMUDANCA_SAL
LEFT JOIN CTE_Hist_Equipe AS EQP 
       ON S.CODCOLIGADA = EQP.CODCOLIGADA 
      AND S.CHAPA       = EQP.CHAPA 
      AND S.DATA       >= EQP.DTMUDANCA 
      AND S.DATA        < EQP.PROXIMA_DATAMUDANCA_EQP
LEFT JOIN CTE_Hist_Ponto AS PON
       ON S.CODCOLIGADA = PON.CODCOLIGADA 
      AND S.CHAPA       = PON.CHAPA 
      AND S.DATA       >= PON.DATAINICIO 
      AND S.DATA        < PON.PROXIMA_DATAINICIO_PON
LEFT JOIN ppessoa AS PP
       ON S.CODPESSOA = PP.CODIGO
LEFT JOIN pcodinstrucao AS PCI
       ON PP.GRAUINSTRUCAO = PCI.CODCLIENTE
LEFT JOIN pcodnacao AS PN 
       ON PP.NACIONALIDADE = PN.CODCLIENTE
""")

# ================================
# Etapa 3: Correção de datas inválidas (mesma regra)
# ================================
data_minima = to_date(lit("1900-01-01"))
data_padrao = to_date(lit("2000-01-01"))

df_corrigido = (
    df_resultado
      .withColumn("DTNASCIMENTO", when(col("DTNASCIMENTO") < data_minima, data_padrao).otherwise(col("DTNASCIMENTO")))
      .withColumn("DATAADMISSAO", when(col("DATAADMISSAO") < data_minima, data_padrao).otherwise(col("DATAADMISSAO")))
      .withColumn("DATADEMISSAO", when(col("DATADEMISSAO") < data_minima, data_padrao).otherwise(col("DATADEMISSAO")))
)

# ================================
# Etapa 4: Escrita no Lakehouse
# ================================
# (Opcional) Reparticionar por chaves que você consulta frequentemente (melhora leitura posterior)
df_corrigido = df_corrigido.repartitionByRange(200, "CODCOLIGADA", "CHAPA")

df_corrigido.write.format("delta").mode("overwrite").save(path_destino)



StatementMeta(, 3c5136a7-54c3-4815-b077-f5e6c4cfcb9a, 3, Finished, Available, Finished)